In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
#Global variables
sampling_rate=512;

In [3]:
data=pd.read_json("../all_data_Swiss-Short.json",orient="records")
[data.columns]

[Index(['fileID', 'sez_len', 'elec_no', 'delta', 'theta', 'alpha', 'beta',
        'lgamma', 'hgamma'],
       dtype='object')]

In [4]:
def mean_std(arr):
    return np.mean(arr),np.std(arr)

In [5]:
def meanning(i,band):
    tmp=data.loc[i,band]
    no_datapoints = np.shape(tmp)[0]
    if data.loc[i,'sez_len'] == (no_datapoints/sampling_rate) - (2*3*60):
        rg=tmp[(3*60*sampling_rate) : (no_datapoints - (3*60*sampling_rate))] #Seizure Segment
        if (len(rg)/sampling_rate)== data.loc[i,'sez_len']:
            pre_mean,pre_std=mean_std(tmp[:(3*60*sampling_rate)])
            sez_mean,sez_std=mean_std(rg)
            post_mean,post_std=mean_std(tmp[no_datapoints - (3*60*sampling_rate):])
            
    pattern = r'\d+'
    matches = re.findall(pattern, data.loc[i,'fileID'])
    pat_id,sez_id= [int(match) for match in matches]
    return data.loc[i,'fileID'],pat_id,sez_id,band,data.loc[i,'sez_len'],data.loc[i,'elec_no'],pre_mean,pre_std,sez_mean,sez_std,post_mean,post_std

In [6]:
# Create an empty DataFrame with the desired columns
df = pd.DataFrame(columns=['file_ID','pat_id','sez_id','band','sez_length','no_of_elec','pre_mean','pre_cv',    'sez_mean','sez_cv','post_mean','post_cv'])

for i in data.index:
    for band in ['delta', 'theta', 'alpha', 'beta','lgamma', 'hgamma']:
        all=meanning(i,band)
        row={
            'file_ID' : all[0],
            'pat_id' : all[1],
            'sez_id' : all[2],
            'band' : all[3],
            'sez_length' : all[4],
            'no_of_elec' : all[5],
            'pre_mean' : all[6],
            'pre_cv' : all[7]/all[6],
            'sez_mean' : all[8],
            'sez_cv' : all[9]/all[8],
            'post_mean' : all[10],
            'post_cv' : all[11]/all[10]
        }
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
        

In [9]:
df.to_json("../all_mean_Swiss-Short.json", orient='records')

In [7]:
# result = df.to_json(orient="records")
# parsed = json.loads(result)
# with open("../all_mean_Swiss-Short.json", "w") as write_file:
#         json.dump(parsed, write_file, indent=4)